# modernmt
> Library to use ModernMT machine translation services

In [ ]:
#| default_exp modernmt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| export
import requests, uuid
import os
from langcodes import *

class modernmttranslate:
    """
    Class to get translations from the ModernMT API
    """
    def __init__(self):
        """Constructor of modernmttranslate class"""
        self._subscription_key = os.getenv('MMT_API_KEY')
        self._endpoint = "https://api.modernmt.com/translate"
        self._languages_cache = []
        
    def check_langpair(self, sourcelang, targetlang):
        """Function to verify if a language pair identified by language ids is supported
        
        Parameters
        ----------
        sourcelang : str
            Source language identifier (BCP-47 format).
        targetlang : str
            Target language identifier (BCP-47 format).
            
        Returns
        -------
        bool
            Returns `True` if language pair is supported, otherwise `False`.
        
        """
        path = '/languages'
        constructed_url = self._endpoint + path
        
        supported_languages = self._languages_cache
        # Cached language array empty if not initialized yet
        if not supported_languages:
            # Not passing API key here, because authentication isn't required for this API endpoint
            request = requests.get(constructed_url)
            response = request.json()
            supported_languages = response["data"]
            self._languages_cache = supported_languages
            
        language_pair_supported = False
        if tag_is_valid(sourcelang) and tag_is_valid(targetlang):
            if sourcelang in supported_languages and targetlang in supported_languages:
                language_pair_supported = True
        
        return language_pair_supported

    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language
        
        Parameters
        ----------
        sourcelang : str
            Source language identifier (BCP-47 format).
        targetlang : str
            Target language identifier (BCP-47 format).
        text : str
            Source text that is to be translated.
            
        Returns
        -------
        str
            Translated text.
        """
        translated_text = ""
        if self.check_langpair(sourcelang,targetlang):
            # Add your subscription key and endpoint
            subscription_key = self._subscription_key
            endpoint = self._endpoint

            # For single text translation ModernMT doesn't need a sub path
            path = ""
            params = {
                'source': sourcelang,
                'target': targetlang,
                'q': text
            }
            constructed_url = endpoint + path

            headers = {
                'X-HTTP-Method-Override': 'GET',
                'Content-Type' : 'application/json',
                'MMT-ApiKey' : self._subscription_key
            }

            request = requests.post(constructed_url, params=params, headers=headers)
            response = request.json()
            translated_text = response["data"]["translation"]

        return translated_text

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
modernmt = modernmttranslate()
translation = modernmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una bella macchina!"

assert True == modernmt.check_langpair("en","it")
assert False == modernmt.check_langpair("en","gu")

In [ ]:
show_doc(modernmttranslate.check_langpair)

In [ ]:
show_doc(modernmttranslate.translate_text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()